In [1]:
import dwavebinarycsp as dbc
import minorminer
from dwave.system.samplers import DWaveSampler 
import statistics as stat
my_solver = 'DW_2000Q_2'
my_token = 'PREM-426a488702253a4cd34901800bf165bb55582e3c'

from pseudobinary import PBP

In [41]:
x = PBP(1,-1) + PBP(2, -1) + PBP(3, -1) + PBP(1, 2)*PBP(2) + PBP(1, 2)*PBP(3) + PBP(2, 2)*PBP(3) + PBP(1, 6)*PBP(2)*PBP(3)
y = x.toQuadratic()
print(x)
print(y)
print(y.toBQM())


-1x1+-1x2+2x1x2+-1x3+2x1x3+2x2x3+6x1x2x3
-1.0x1+-1.0x2+8.0x1x2+-1.0x3+8.0x1x3+8.0x2x3+6.0x4+-6.0x1x4+-6.0x2x4+-6.0x3x4
BinaryQuadraticModel({4: 6.0, 1: -1.0, 2: -1.0, 3: -1.0}, {(1, 4): -6.0, (2, 4): -6.0, (3, 4): -6.0, (1, 2): 8.0, (1, 3): 8.0, (2, 3): 8.0}, 0, Vartype.BINARY)


In [8]:
def print_list(lst, header='', cpad=6, rpad=120):
    line = ''
    for i in range(len(lst))[::-1]:
        if lst[i]:
            line += str(lst[i]).rjust(cpad)
        else:
            line += ''.rjust(cpad)
    line = header.ljust(cpad) + line.rjust(rpad-cpad)
    print(line)

modulus = 143 #60491
N = list()
while modulus > 0:
    N.append(PBP(0,-(modulus%2)))
    modulus >>= 1

n = len(N)//2 - 2
p = [PBP()] + [PBP(x) for x in range(1,n+1)] + [PBP()]
q = [PBP()] + [PBP(x) for x in range(n+1,2*n+1)] + [PBP()]
print_list(p, 'p')
print_list(q, 'q')

next_var = 2*n+1

mtab = list()
for i in range(len(q)):
    row = [None]*i
    mtab.append(row)
    for pi in p: row.append(pi*q[i])

for i in range(1, len(p)+len(q)-1):
    count = 0
    for row in mtab:
        if len(row) > i and row[i]: count += 1
    carries = math.floor(math.log2(count))
    carry_mult = 2
    for j in range(carries):
        row = [None] * i
        row.append(PBP(next_var, -carry_mult))
        row += [None] * j
        row.append(PBP(next_var))
        next_var += 1
        carry_mult *= 2
        mtab.append(row)
        
mtab.append(N)
        
for row in mtab: print_list(row)

p                                                                                                    1    x2    x1     1
q                                                                                                    1    x4    x3     1
                                                                                                     1    x2    x1     1
                                                                                              x3  x2x3  x1x3    x3      
                                                                                        x4  x2x4  x1x4    x4            
                                                                                   1    x2    x1     1                  
                                                                                                          x5  -2x5      
                                                                                                    x6  -2x6            
                                

In [28]:
eqs = list()
for j in range(1,len(N)):
    eqn = PBP(0,0)
    for row in mtab:
        if len(row)>j and row[j]: eqn += row[j]
    eqs.append(eqn)

for eqn in eqs: print(eqn)
    
print("\neliminate x5")
eqs2 = list()
for eqn in eqs: eqs2.append(eqn.substitute(5, PBP(0,0)))
for eqn in eqs2: print(eqn)
    
print("\neliminate x1")
eqs3 = list()
for i in range(1,len(eqs2)): eqs3.append(eqs2[i].substitute(1, PBP(0,1)+PBP(3,-1)))
for eqn in eqs3: print(eqn)
    
print("\neliminate x14")
eqs4 = list()
for i in range(0,len(eqs3)-1): eqs4.append(eqs3[i].substitute(14, PBP(0,1)+PBP(13,-1)))
for eqn in eqs4: print(eqn)
    
print("\neliminate x6")
eqs5 = list()
for eqn in eqs4: eqs5.append(eqn.substitute(6, PBP(0,0)))
for eqn in eqs5: print(eqn)

print("\neliminate x7")
eqs6 = list()
for eqn in eqs5: eqs6.append(eqn.substitute(7, PBP(0,0)))
for eqn in eqs6: print(eqn)
    
print("\neliminate x2")
eqs7 = list()
for i in range(1,len(eqs6)): eqs7.append(eqs6[i].substitute(2, PBP(0,1)+PBP(4,-1)))
for eqn in eqs7: print(eqn)
    
H = PBP(0,0)
for eqn in eqs7: H += eqn*eqn
#print(H)
H = H.toQuadratic()
bqm = H.toBQM()
#print(bqm)

-1+x1+x3+-2x5
-1+-2x6+x2+-4x7+x1x3+x4+x5
1+x6+-2x8+-4x9+x2x3+x1x4
0+x1+x7+x8+x3+-2x10+-4x11+x2x4
0+x9+x2+x10+-2x12+-4x13+x4
1+x11+x12+-2x14
-1+x13+x14

eliminate x5
-1+x1+x3
-1+-2x6+x2+-4x7+x1x3+x4
1+x6+-2x8+-4x9+x2x3+x1x4
x7+x1+x8+-2x10+x3+-4x11+x2x4
x9+x10+x2+-2x12+-4x13+x4
1+x11+x12+-2x14
-1+x13+x14

eliminate x1
-1+-2x6+x2+-4x7+x4
1+x6+-2x8+-4x9+x2x3+x4+-1x3x4
1+-2x10+x7+x8+-4x11+x2x4
x9+x10+x2+-2x12+-4x13+x4
1+x11+x12+-2x14
-1+x13+x14

eliminate x14
-1+-2x6+x2+-4x7+x4
1+x6+-2x8+-4x9+x2x3+x4+-1x3x4
1+-2x10+x7+x8+-4x11+x2x4
x9+x10+x2+-2x12+-4x13+x4
-1+x11+2x13+x12

eliminate x6
-1+x4+x2+-4x7
1+-2x8+-4x9+x2x3+x4+-1x3x4
1+-2x10+x7+x8+-4x11+x2x4
x9+x10+x2+-2x12+-4x13+x4
-1+x11+2x13+x12

eliminate x7
-1+x4+x2
1+-2x8+-4x9+x2x3+x4+-1x3x4
1+-2x10+x8+-4x11+x2x4
x9+x10+x2+-2x12+-4x13+x4
-1+x11+2x13+x12

eliminate x2
1+-2x8+-4x9+x3+x4+-2x3x4
1+-2x10+x8+-4x11
1+x9+-4x13+x10+-2x12
-1+x11+2x13+x12


In [29]:
sampler = DWaveSampler(solver=my_solver, token=my_token)
_, target_edgelist, target_adjacency = sampler.structure

embedding = minorminer.find_embedding(bqm.quadratic, target_edgelist)
if bqm and not embedding:
    raise ValueError("no embedding found")
    
bqm_embedded = dimod.embed_bqm(bqm, embedding, target_adjacency, 8)

print(len(list(embedding.keys())), "logical qubits")
phys = list()
for _, v in embedding.items():
    phys += v
print(len(phys), "physical qubits")

chain_lens = [len(v) for _, v in embedding.items()]
print('Longest chain: ', max(chain_lens))
print('Shortest chain: ', min(chain_lens))
print('Average chain: ', stat.mean(chain_lens))

response = sampler.sample(bqm_embedded, num_reads = 1000, answer_mode = 'histogram')
response = dimod.unembed_response(response, embedding, source_bqm=bqm)

row_format ="{:>8}" * 2
print(row_format.format('Energy', 'Count'))
for sample in response.data(fields=['sample', 'energy', 'num_occurrences']):
    print(row_format.format(sample[1], sample[2]))
    print(sample[0])

10 logical qubits
22 physical qubits
Longest chain:  4
Shortest chain:  1
Average chain:  2.2
  Energy   Count
     0.0      98
{3: 1, 4: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1, 15: 1}
     0.0      52
{3: 0, 4: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1, 15: 0}
     0.0      54
{3: 1, 4: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1, 15: 0}
     0.0      59
{3: 0, 4: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1, 15: 1}
     1.0      45
{3: 1, 4: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1, 15: 1}
     1.0      11
{3: 0, 4: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1, 15: 0}
     1.0       6
{3: 0, 4: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
     2.0       5
{3: 0, 4: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
     2.0      83
{3: 1, 4: 1, 8: 0, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1, 15: 1}
     3.0       1
{3: 1, 4: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 1, 15: 1}
     3.0      18
{3: 0, 4: 0, 8: 0, 9: 0, 10: 0, 

In [27]:
#tests
print(PBP(), " should be 1")
print(PBP(0,2), " should be 2")
print(PBP(0,-2), " should be -2")
print(PBP(1), " should be x1")
print(PBP(1,2), " should be 2x1")
print(PBP(15), " should be x1x2x3x4")
print(PBP(0,2)*PBP(15), "should be 2x1x2x3x4")
print(PBP(2)*PBP(4), "should be x2x3")
x1 = PBP(1)
x2 = PBP(2)
print(x1, " should be x1")
print(x2, " should be x2")
print(x1+x2, "should be x1+x2")
print((x1+x2)*(x1+x2), "should be x1+x2+2x1x2")

1  should be 1
2  should be 2
-2  should be -2
x1  should be x1
2x1  should be 2x1
x1x2x3x4  should be x1x2x3x4
2x1x2x3x4 should be 2x1x2x3x4
x2x3 should be x2x3
x1  should be x1
x2  should be x2
x1+x2 should be x1+x2
x1+x2+2x1x2 should be x1+x2+2x1x2


In [49]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list() -> new empty list
 |  list(iterable) -> new list initialized from iterable's items
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __l